In [1]:
import comtradeapicall
import json
import pandas as pd
import time

In [43]:
# your UN comtrade API
key = ''

In [3]:
#country list
with open("../data/countries.json", "r", encoding="utf-8") as f:
    data = json.load(f)


In [4]:
country_df = pd.DataFrame(data['results'])


In [5]:
country_df.head()

,id,text,reporterCode,reporterDesc,reporterNote,reporterCodeIsoAlpha2,reporterCodeIsoAlpha3,entryEffectiveDate,isGroup,entryExpiredDate
0,4,Afghanistan,4,Afghanistan,Afghanistan,AF,AFG,1900-01-01T00:00:00,False,NaN
1,8,Albania,8,Albania,Albania,AL,ALB,1900-01-01T00:00:00,False,NaN
2,12,Algeria,12,Algeria,Algeria,DZ,DZA,1900-01-01T00:00:00,False,NaN
3,20,Andorra,20,Andorra,Andorra,AD,AND,1900-01-01T00:00:00,False,NaN
4,24,Angola,24,Angola,Angola,AO,AGO,1900-01-01T00:00:00,False,NaN


In [6]:
country_list = country_df.loc[country_df["isGroup"] == False, "id"].tolist()
country_list = [str(country) for country in country_list]
all_country = ",".join(country_list)

In [10]:
metal_list = ["283691", "282520", "7502", "8101", "8102", "8103", "8104", "8105", "8106", "8108", "8109", "8110", "8111", "7106", "7108", "7110"]

In [11]:
metal_dic = {
    "8101":"Tungsten",
    "8102":"Molybdenum",
    "8103":"Tantalum", 
    "8104":"Magnesium", 
    "8105":"Cobalt",
    "8106":"Bismuth",
    "8108":"Titanium",
    "8109":"Zirconium",
    "8110":"Antimony", 
    "8111":"Manganese",
    "7106":"Silver", 
    "7108":"Gold", 
    "7110":"Platinum",
    "7502": "Nickel",
    "283691": "Lithium carbonate",
    "282520": "Lithium hydroxide"
}

In [7]:
df = comtradeapicall.previewFinalData(
    typeCode='C',
    freqCode='A',
    clCode='HS',
    reporterCode=all_country,
    partnerCode=all_country,
    partner2Code=0,
    customsCode=None, 
    motCode='0',
    period='2024',
    cmdCode='282520',
    flowCode='X'
)

print(df.head())

  typeCode freqCode  refPeriodId  refYear  refMonth period  reporterCode  \
0        C        A     20240101     2024        52   2024            36   
1        C        A     20240101     2024        52   2024            36   
2        C        A     20240101     2024        52   2024            40   
3        C        A     20240101     2024        52   2024            40   
4        C        A     20240101     2024        52   2024            40   

  reporterISO reporterDesc flowCode  ...       netWgt  isNetWgtEstimated  \
0        None         None        X  ...  1316630.000              False   
1        None         None        X  ...   936554.000              False   
2        None         None        X  ...       45.000              False   
3        None         None        X  ...        2.855               True   
4        None         None        X  ...       15.081               True   

   grossWgt isGrossWgtEstimated  cifvalue      fobvalue  primaryValue  \
0  1339.100  

In [8]:
df.to_csv("../data/sample_all_to_all_preview.csv")

In [22]:
getting_year = [str(year) for year in range(2015, 2025)]

In [21]:
for year in getting_year:
    df_list = []
    for commodity in metal_list:
        time.sleep(0.5)
        df = comtradeapicall.getFinalData(
            subscription_key=key,
            typeCode='C',
            freqCode='A',
            clCode='HS',
            reporterCode=all_country,
            partnerCode=all_country,
            partner2Code=0,
            customsCode=None,
            motCode='0',
            period= year,
            cmdCode=commodity,
            flowCode='X'
        )
        if not df.empty:
            df_list.append(df)

    trade_df = pd.concat(df_list, ignore_index=True)
    trade_df.to_csv(f'../data/trade{year}.csv')
    


In [24]:
csv_files = [f'../data/trade{year}.csv' for year in getting_year]
trade_dfs = [pd.read_csv(f) for f in csv_files]
combined_trade_df = pd.concat(trade_dfs, ignore_index=True)

In [25]:
combined_trade_df['cmdCode'] = combined_trade_df['cmdCode'].astype('str')
combined_trade_df['period'] = combined_trade_df['period'].astype('str')

In [26]:
combined_trade_df['source'] = combined_trade_df.apply(lambda x: country_df.loc[country_df['id'] == x['reporterCode'], 'text'].iloc[0], axis=1)
combined_trade_df['target'] = combined_trade_df.apply(lambda x: country_df.loc[country_df['id'] == x['partnerCode'], 'text'].iloc[0], axis=1)
combined_trade_df['product'] = combined_trade_df.apply(lambda x: metal_dic[x['cmdCode']], axis=1)

In [27]:
complete_df = combined_trade_df[['period', 'product', 'source', 'target', 'primaryValue']].rename(columns = {
    'period':'year',
    'primaryValue': 'value'
    })

In [28]:
complete_df

,year,product,source,target,value
0,2015,Lithium carbonate,Argentina,Australia,4.059137e+05
1,2015,Lithium carbonate,Argentina,Belgium,3.108146e+06
2,2015,Lithium carbonate,Argentina,Chile,2.475500e+04
3,2015,Lithium carbonate,Argentina,China,1.237741e+07
4,2015,Lithium carbonate,Argentina,Colombia,5.738000e+03
...,...,...,...,...,...
179957,2024,Platinum,Türkiye,South Africa,1.201000e+03
179958,2024,Platinum,Türkiye,Switzerland,4.532209e+06
179959,2024,Platinum,Türkiye,United Kingdom,3.719000e+03
179960,2024,Platinum,Türkiye,USA,3.549010e+05


In [29]:
complete_df.to_csv('../data/tradedata.csv')